## Problem statement

The ECG is a time series that measures the electrical activity of the heart. This is the main tool to diagnose heart diseases. Recording an ECG is simple: 3 electrodes are placed at the ends of limbs, and 6 on the anterior chest.This generates 12 time series, called leads, each corresponding to a difference in potential between a pair of electrodes.
The electrodes’ position is very important to correctly interpret the ECG. Making the mistake of inverting electrodes compromises interpretation, either because the leads do not explore the expected area (errors in the measures of hypertrophia indices, in the analysis of the ST segment), or because they generate false abnormalities (fake Q waves, error in the heart’s axis…).
Inversion errors are frequent (5% of ECGs), and only experts (cardiologists) manage to detect them. But most ECGs are not interpreted by experts: only 30% are, the rest being interpreted by nurses or general practitioners. An algorithm for automatic detection of electrode inversion is therefore paramount to the correct interpretation of ECGs and would improve the quality of diagnosis.
This project is intended to make you detect electrode inversion in an ECG. The dataset at your disposal contains ECGs from a cardiology center. An ECG will be labeled as correctly realised (0) or as inverted (1). The goal is to perform binary classification on these ECGs.

The key objective of this homework is to propose a model relevant to the task that shows good accuracy in detection of lead inversion.

In [ ]:
# Just defining some standard imports and configs I like to use when working with notebooks

from __future__ import absolute_import, division, print_function, unicode_literals

%load_ext autoreload
%autoreload 2

import sys
assert sys.version_info.major == 3, 'Not running on Python 3'

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import logging
logging.basicConfig(level=logging.INFO, stream=sys.stdout)

## Data

Data loading from pickled file.
Data:
- The training data contains 1400 ECGs and their labels. For each ECG, the data consists of 10 seconds of recording for 12 leads, each sampled at 250Hz.
- The testing data contains 2630 ECGs.
Each input file therefore contains the ECG signal in the form (n_ecgs, n_samples=2500, n_leads=12).

We got for the training set 1400 examples of a 10 seconds ECG (2500 steps sampled at 250Hz) for the 12 leads. The 3-dimensional vector 

In [ ]:
import pandas as pd
import os.path
import ast
import wfdb
import numpy as np

In [ ]:
path = 'data/ptb-xl/'
sampling_rate = 100

class_dict = {'NORM': 0,
              'LAFB': 1, 'LPFB': 1, 'IRBBB': 1, 'CLBBB': 1, 'CRBBB': 1, 'IVCD': 1, '_AVB': 1, 'WPW': 1, 'ILBBB': 1,
              'STTC': 2, 'NST_': 2, 'ISCA': 2, 'ISC_': 2, 'ISCI': 2,
              'AMI': 3, 'IMI': 3, 'LMI': 3,
              'LVH': 4, 'LAO': 4, 'LAE': 4, 'RAO': 4, 'RAE': 4,
              }


def load_raw_data(df, _sampling_rate, _path, drop):
    if _sampling_rate == 100:
        data = [wfdb.rdsamp(_path+f) for i, f in enumerate(df.filename_lr) if i not in drop]
    else:
        data = [wfdb.rdsamp(_path+f) for i, f in enumerate(df.filename_hr) if i not in drop]
    data = [signal for signal, meta in data]
    return np.array(data)


ds_x = pd.read_csv(os.path.join(path, 'ptbxl_database.csv'), index_col='ecg_id')
y_train_raw = [max(ast.literal_eval(code), key=ast.literal_eval(code).get)
           if max(ast.literal_eval(code).values()) > 20 else 'NONE'
           for code in ds_x.scp_codes]

# Cleaning-up unknown labels
to_drop = [i for i, c in enumerate(y_train_raw) if c not in class_dict]

x_train_raw = load_raw_data(ds_x, sampling_rate, path, drop=to_drop)[:1400]
y_train_raw = np.array([class_dict[c] for i, c in enumerate(y_train_raw) if i not in to_drop])[:1400]
y_train_raw[y_train_raw > 0] = 1

print('Training dataset shape: {} - Training labels shape: {}'.format(
    x_train_raw.shape, y_train_raw.shape))

In [ ]:
import seaborn as sns
sns.set_context('notebook')
sns.set(style="whitegrid", font_scale=1.5)
sns.despine()

import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
plt.rcParams['figure.figsize'] = [20, 10]

In [ ]:
fig, ax = plt.subplots(x_train_raw.shape[-1], 1, sharex=True, sharey=True, figsize=[20, 10])
for i in range(x_train_raw.shape[-1]):
    sns.lineplot(data=x_train_raw[0, :, i], ax=ax[i])
plt.show();

for i in range(x_train_raw.shape[-1]):
    sns.lineplot(data=x_train_raw[0, :, i])
plt.show();

In [ ]:
sns.countplot(x=y_train_raw)
plt.show();

# Proposed Solution

....


## Pre-processing

I identified 4 pre-processing steps looking at some artcles: denoising, drift removal, resampling, normalization. 
1. For denoising and drift removal everyone seems to use a wavelet based analysis to remove high frequency components and a median filter to remove drift below 2Hz.
2. For drift removal
3. I am not sure resampling is relevant in this case since all data should be already at the same frequency of 250Hz (might obtain improvements with a lower sampling rate? it might reduce noise but lose some information, to be eventually explored). 
4. For normalization I should bring values in the range of [-1, 1], [0, 1] is also an option. I might adopt a simple normalization or a z score normalization (is it better to normalize lead by lead or use the global statics of the entire sample? to be explored if I got time)

In [ ]:
#!pip install pywavelets
import pywt
#!pip install biosppy
from biosppy.signals.tools import filter_signal

In [ ]:
def wavelet_denoising(x, wavelet='db4', level=1):
    coeff = pywt.wavedec(x, wavelet, mode="per", level=level)
    #coeff[0] = np.array([0 if i != 6 else coeff[0][i] for i in range(len(coeff[0]))])

    #keep = [0, level - 6 + 1, level - 5 + 1, level - 4 + 1]
    #for i in range(len(coeff)):
    #    if i not in keep:
    #        coeff[i] = np.zeros(len(coeff[i]))

    sigma = np.mean(np.absolute(coeff[-1] - np.mean(coeff[-1]))) / 0.6745
    uthresh = sigma * np.sqrt(2 * np.log(len(x)))
    coeff[1:] = (pywt.threshold(i, value=uthresh, mode='hard') for i in coeff[1:])
    return pywt.waverec(coeff, wavelet, mode='per')

# db4 or db5 seems to be the wavelets better suited to ecg analysis.
# Starting with db4, might explore if time remaining using db5

wav = pywt.Wavelet('db4')
sns.lineplot(data=x_train_raw[0, :, 11], label='Raw')

x_train_denoised = np.zeros(x_train_raw.shape)
level = int(np.log2((0.6745 * 100) / 0.5))
for s in range(x_train_raw.shape[0]):
    for der in range(x_train_raw.shape[-1]):
        x_train_denoised[s, :, der] =  wavelet_denoising(x_train_raw[s, :, der], wavelet=wav,
                                                         level=level)

sns.lineplot(data=x_train_denoised[0, :, 11], label='Wavelet Denoising db4 - {}'.format(level))
plt.show();

I did not find satisfying results removeing the low frequency components of the approximate coefficients (might be doing it wrong?) Thresholding based on the mean absolute deviation of the first detailed coefficient seems promising.


In [ ]:
sns.lineplot(data=x_train_denoised[0, :, 11], label='Wavelet Denoising db4 - {}'.format(level))

order = int(0.3 * sampling_rate)

x_train_nodrift = np.zeros(x_train_denoised.shape)
for s in range(x_train_denoised.shape[0]):
    for der in range(x_train_denoised.shape[-1]):
        x_train_nodrift[s, :, der], _, _ = filter_signal(signal=x_train_denoised[s, :, der], ftype="FIR", band="bandpass",
                                                 order=order, frequency=[2.5, 40],
                                                 sampling_rate=sampling_rate)

sns.lineplot(data=x_train_nodrift[0, :, 11], label='Baseline Drift Removed - Bandpass order {}'. format(order))
plt.show();

In [ ]:
def z_score_norm(x):
    mean = np.mean(x)
    std = np.std(x)
    if std > 0:
        x = (x - mean) / std
    else:
        x *= 0.
    return x

x_train_centered = np.zeros(x_train_denoised.shape)
for s in range(x_train_nodrift.shape[0]):
    for der in range(x_train_nodrift.shape[-1]):
        x_train_centered[s, :, der] = z_score_norm(x_train_nodrift[s, :, der])
        
sns.lineplot(data=x_train_centered[0, :, 11], label='Baseline Drift Removed')
plt.show();

In [ ]:
from sklearn.model_selection import StratifiedKFold

skf = StratifiedKFold(n_splits=5, random_state=42, shuffle=True)
skf.get_n_splits(x_train_centered, y_train_raw)

train_folds = []
val_folds = []
for train_index, test_index in skf.split(x_train_centered, y_train_raw):
    train_folds.append((x_train_centered[train_index], y_train_raw[train_index]))
    val_folds.append((x_train_centered[test_index],  y_train_raw[test_index]))

# For the moment let's just work on one fold 
train_ds_fold0 = train_folds[0]
val_ds_fold0 = val_folds[0]

For the deep learning framework I decided to use Tensorflow 2.x (2.8 in my case). Can work on gpu using memory growth strategy and possibility of enabling eager debugging for @tf.functions. Tensorboard logs are stored in the ./tf_logs directory

In [ ]:
use_gpu = True
debug_mode = False # Reminder to myself: set to false before sending 

# Tensorflow 2.x standard import + tensorboard
from packaging import version
import tensorflow as tf
print("TensorFlow version: {}".format(tf.__version__))
assert version.parse(tf.__version__).release[0] >= 2, "This notebook requires TensorFlow 2.0 or above."
%load_ext tensorboard
from tensorboard.plugins.hparams import api as hp

# GPU accelerated processing
if use_gpu:
    gpu_list = tf.config.list_physical_devices('GPU')
    print("GPUs Available:")
    if gpu_list:
        try:
            for gpu in gpu_list:
                tf.config.experimental.set_memory_growth(gpu, True)
                logical_gpus = tf.config.experimental.list_logical_devices('GPU')
                print(len(gpu_list), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
        except RuntimeError as e:
            print(e)
    else:
        print('None')

# Debugging
tf.config.run_functions_eagerly(debug_mode)
if debug_mode:
    tf.data.experimental.enable_debug_mode()
print("Eager execution: {}".format(tf.executing_eagerly()))


# Empty logs and cache
!rm -rf ./tf_logs/ 
tf.keras.backend.clear_session()

In [ ]:
epochs = 20
batch_size = 64
initial_lr = 1e-6


In [ ]:
train_ds = tf.data.Dataset.from_tensor_slices(train_ds_fold0)
val_ds = tf.data.Dataset.from_tensor_slices(val_ds_fold0)

train_ds = train_ds.batch(batch_size, num_parallel_calls=tf.data.experimental.AUTOTUNE,
                                deterministic=False).prefetch(tf.data.experimental.AUTOTUNE
                                                             ).cache().shuffle(len(train_ds),
                                                                               reshuffle_each_iteration=True)
val_ds = val_ds.batch(batch_size, num_parallel_calls=tf.data.experimental.AUTOTUNE,
                            deterministic=True).prefetch(tf.data.experimental.AUTOTUNE).cache()

In [ ]:
class ConvBlock(tf.keras.layers.Layer):
    def __init__(self, filters=64, kernel_size=16, use_final_conv=True, use_final_pooling=True,
                 name='ConvBlock', **kwargs):
        super(ConvBlock, self).__init__(name=name, **kwargs)
        self.use_final_conv = use_final_conv
        self.use_final_pooling = use_final_pooling

        self.batch_1 = tf.keras.layers.BatchNormalization(axis=-1)
        self.act_1 = tf.keras.layers.ReLU()
        self.drop_1 = tf.keras.layers.Dropout(0.5)
        self.conv_1 = tf.keras.layers.Conv1D(filters, kernel_size=kernel_size, padding='same', 
                                             kernel_initializer='he_normal', activation=None)
        self.batch_2 = tf.keras.layers.BatchNormalization(axis=-1)
        self.act_2 = tf.keras.layers.ReLU()
        self.drop_2 = tf.keras.layers.Dropout(0.5)
        self.conv_2 = tf.keras.layers.Conv1D(filters, kernel_size=kernel_size, padding='same',
                                             kernel_initializer='he_normal', activation=None)
        
        self.pool_fin = tf.keras.layers.MaxPooling1D(pool_size=2, strides=2)
        
        self.conv_par = tf.keras.layers.Conv1D(filters, kernel_size=1)
        self.pool_par = tf.keras.layers.MaxPooling1D(pool_size=2, strides=2)
        
    @tf.function
    def call(self, inputs, training=None, **kwargs):
        if self.use_final_conv:
            xshort = self.conv_par(inputs, training=training)
        else:
            xshort = inputs
        
        x1 = self.batch_1(inputs, training=training)
        x1 = self.act_1(x1, training=training)
        x1 = self.drop_1(x1, training=training)
        x1 = self.conv_1(x1, training=training)
        x1 = self.batch_2(x1, training=training)
        x1 = self.act_2(x1, training=training)
        x1 = self.drop_2(x1, training=training)
        x1 = self.conv_2(x1, training=training)
        
        if self.use_final_pooling:
            x1 = self.pool_fin(x1, training=training)
            x2 = self.pool_par(xshort, training=training)
        else:
            x2 = xshort
        
        return x1 + x2

        
class ResNet1D(tf.keras.models.Model):
    def __init__(self, base_filters=64, kernel_size = 16, n_class=2, inshape=(1000,12)):
        super(ResNet1D, self).__init__(self)

        '-----------------------'
        self.conv_pre = tf.keras.layers.Conv1D(base_filters, kernel_size=kernel_size, padding='same', 
                                               kernel_initializer='he_normal', activation=None,
                                               input_shape=inshape)
        self.batch_pre = tf.keras.layers.BatchNormalization(axis=-1)
        self.act_pre = tf.keras.layers.ReLU()
        '-----------------------'
        self.conv_l1 = tf.keras.layers.Conv1D(base_filters, kernel_size=kernel_size, padding='same',
                                              kernel_initializer='he_normal', activation=None)
        self.batch_l1 = tf.keras.layers.BatchNormalization(axis=-1)
        self.act_l1 = tf.keras.layers.ReLU()
        self.drop_l1 = tf.keras.layers.Dropout(0.5)
        self.conv_l2 = tf.keras.layers.Conv1D(base_filters, kernel_size=kernel_size, padding='same', 
                                              kernel_initializer='he_normal', activation=None)
        self.pool_l1 = tf.keras.layers.MaxPooling1D(pool_size=2, strides=2)
        
        self.pool_r1 = tf.keras.layers.MaxPooling1D(pool_size=2, strides=2)
        '-----------------------'
        self.conv_block1 = ConvBlock(filters=base_filters * 1, kernel_size=kernel_size,
                                     use_final_conv=False, use_final_pooling=False)
        self.conv_block2 = ConvBlock(filters=base_filters * 1, kernel_size=kernel_size,
                                     use_final_conv=False, use_final_pooling=True)
        self.conv_block3 = ConvBlock(filters=base_filters * 1, kernel_size=kernel_size,
                                     use_final_conv=False, use_final_pooling=False)
        self.conv_block4 = ConvBlock(filters=base_filters * 1, kernel_size=kernel_size,
                                     use_final_conv=False, use_final_pooling=True)
        self.conv_block5 = ConvBlock(filters=base_filters * 2, kernel_size=kernel_size,
                                     use_final_conv=True, use_final_pooling=False)
        self.conv_block6 = ConvBlock(filters=base_filters * 2, kernel_size=kernel_size,
                                     use_final_conv=False, use_final_pooling=True)
        self.conv_block7 = ConvBlock(filters=base_filters * 2, kernel_size=kernel_size,
                                     use_final_conv=False, use_final_pooling=False)
        self.conv_block8 = ConvBlock(filters=base_filters * 2, kernel_size=kernel_size,
                                     use_final_conv=False, use_final_pooling=True)
        self.conv_block9 = ConvBlock(filters=base_filters * 3, kernel_size=kernel_size,
                                     use_final_conv=True, use_final_pooling=False)
        self.conv_block10 = ConvBlock(filters=base_filters * 3, kernel_size=kernel_size,
                                      use_final_conv=False, use_final_pooling=True)
        self.conv_block11 = ConvBlock(filters=base_filters * 3, kernel_size=kernel_size,
                                      use_final_conv=False, use_final_pooling=False)
        self.conv_block12 = ConvBlock(filters=base_filters * 3, kernel_size=kernel_size,
                                      use_final_conv=False, use_final_pooling=True)
        self.conv_block13 = ConvBlock(filters=base_filters * 4, kernel_size=kernel_size,
                                      use_final_conv=True, use_final_pooling=False)
        self.conv_block14 = ConvBlock(filters=base_filters * 4, kernel_size=kernel_size,
                                      use_final_conv=False, use_final_pooling=True)
        self.conv_block15 = ConvBlock(filters=base_filters * 4, kernel_size=kernel_size,
                                      use_final_conv=False, use_final_pooling=False)
        '-----------------------'
        self.batch_final = tf.keras.layers.BatchNormalization(axis=-1)
        self.act_final = tf.keras.layers.ReLU()
        self.f = tf.keras.layers.Flatten()
        self.dense_final = tf.keras.layers.Dense(n_class, activation='softmax')
        

    @tf.function()
    def call(self, inputs, training=None):
        x = self.conv_pre(inputs, training=training)
        x = self.batch_pre(x, training=training)
        x_pre = self.act_pre(x, training=training)
        
        x_branch_l = self.conv_l1(x_pre, training=training)
        x_branch_l = self.batch_l1(x_branch_l, training=training)
        x_branch_l = self.act_l1(x_branch_l, training=training)
        x_branch_l = self.drop_l1(x_branch_l, training=training)
        x_branch_l = self.conv_l2(x_branch_l, training=training)
        x_branch_l = self.pool_l1(x_branch_l, training=training)
        
        x_branch_r = self.pool_r1(x_pre, training=training)
                
        x_in = x_branch_l + x_branch_r
        
        x_conv = self.conv_block1(x_in, training=training)
        x_conv = self.conv_block2(x_conv, training=training)
        x_conv = self.conv_block3(x_conv, training=training)
        x_conv = self.conv_block4(x_conv, training=training)
        x_conv = self.conv_block5(x_conv, training=training)
        x_conv = self.conv_block6(x_conv, training=training)
        x_conv = self.conv_block7(x_conv, training=training)
        x_conv = self.conv_block8(x_conv, training=training)
        x_conv = self.conv_block9(x_conv, training=training)
        x_conv = self.conv_block10(x_conv, training=training)
        x_conv = self.conv_block11(x_conv, training=training)
        x_conv = self.conv_block12(x_conv, training=training)
        x_conv = self.conv_block13(x_conv, training=training)
        x_conv = self.conv_block14(x_conv, training=training)
        x_conv = self.conv_block15(x_conv, training=training)
        
        x_final = self.batch_final(x_conv, training=training)
        x_final = self.act_final(x_final, training=training)
        x_final = self.f(x_final)
        x_final = self.dense_final(x_final, training=training)
        
        return x_final


In [ ]:
def get_model(inshape, base_filters, dense_filters, n_convs = 3, n_dense = 2, ):

    model = tf.keras.Sequential()
    m = 1
    for i in range(n_convs):
        if i == 0:
            model.add(tf.keras.layers.Conv1D(base_filters * m, 3, padding='same',
                                             activation='relu', input_shape=inshape))
        else:
            model.add(tf.keras.layers.Conv1D(base_filters * m, 3, padding='same',
                                             activation='relu'))
        model.add(tf.keras.layers.BatchNormalization(axis=-1))
        model.add(tf.keras.layers.MaxPooling1D())
        m += m
    model.add(tf.keras.layers.Flatten())
    if n_dense > 1:
        m = 2 * (n_dense - 1)
    else:
        m = 1
    for i in range(n_dense):
        model.add(tf.keras.layers.Dense(dense_filters * m, activation='relu'))
        m = m // 2
    model.add(tf.keras.layers.Dense(2))

    model.summary()

    return model

#model = get_model((1000, 12), 128, 8, 1, 1)

opt = tf.keras.optimizers.Adam(learning_rate=initial_lr)

loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

metric_fn = tf.keras.metrics.CategoricalAccuracy()

In [ ]:
%tensorboard --logdir ./tf_logs/hparam_tuning --host 0.0.0.0

In [ ]:
HP_EPOCHS = hp.HParam('epochs', hp.Discrete([20]))
HP_BATCH = hp.HParam('batch', hp.Discrete([4]))
HP_LR = hp.HParam('initial_lr', hp.Discrete([1e-6]))
HP_FILTERS = hp.HParam('base_filters', hp.Discrete([64]))
HP_DENSEFILTERS = hp.HParam('dense_filters', hp.Discrete([16]))
HP_N_CONV = hp.HParam('n_conv_filters', hp.Discrete([2]))
HP_N_DENSE = hp.HParam('n_dense_filters', hp.Discrete([3]))

session_num = 0
run_name = 'run'

def hyper_parameters_config():
    for e in HP_EPOCHS.domain.values:
        for lr in HP_BATCH.domain.values:
            for a in HP_LR.domain.values:
                for bf in HP_FILTERS.domain.values:
                    for d in HP_DENSEFILTERS.domain.values:
                        for nc in HP_N_CONV.domain.values:
                            for nd in HP_N_DENSE.domain.values:
                                yield {
                                    HP_EPOCHS: e,
                                    HP_BATCH: lr,
                                    HP_LR: a,
                                    HP_FILTERS: bf,
                                    HP_DENSEFILTERS: d,
                                    HP_N_CONV: nc,
                                    HP_N_DENSE: nd,
                                }

for hparams in hyper_parameters_config():
    run_name = 'run-{}'.format(session_num)
    print('\n\n--- Starting trial: {}'.format(run_name))
    print({h.name: hparams[h] for h in hparams})

    run_logdir = 'tf_logs/hparam_tuning/' + run_name
    summary_writer = tf.summary.create_file_writer(run_logdir)

    with summary_writer.as_default():
        hp.hparams(hparams)
        #model = get_model((1000, 12), hparams[HP_FILTERS], hparams[HP_DENSEFILTERS], hparams[HP_N_CONV], hparams[HP_N_DENSE])
        model = ResNet1D(hparams[HP_FILTERS], hparams[HP_DENSEFILTERS])
        model.compile(optimizer=tf.keras.optimizers.Adam(),
                      loss='sparse_categorical_crossentropy',
                      metrics=['accuracy',],)

        model.fit(x=train_ds, validation_data=val_ds,
                  batch_size=batch_size, epochs=hparams[HP_EPOCHS], verbose=2, 
                  callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, verbose=1),
                             tf.keras.callbacks.TensorBoard(log_dir=run_logdir, histogram_freq=1)],
                  use_multiprocessing=True)
        
        model.summary()


    session_num += 1

In [ ]:
y_pred = np.argmax(model.predict(val_ds_fold0[0]), axis=1)
y_true = val_ds_fold0[1]

test_acc = sum(y_pred == y_true) / len(y_true)
print(f'Test set accuracy: {test_acc:.0%}')

confusion_mtx = tf.math.confusion_matrix(y_true, y_pred)
sns.heatmap(confusion_mtx,
            annot=True, fmt='g')
plt.xlabel('Prediction')
plt.ylabel('Label')
plt.show();